In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("documents/overbond_interview/secondaries.csv")

In [3]:
#Lets start with finding and remove any Nan values and outliers

In [4]:
#Getting sum of all nan values
df.isnull().sum(axis = 0)


Unnamed: 0                      0
Unnamed: 0.1                    0
security_id                     0
amount                          0
issue_date                      0
issue_yield                   122
maturity_date                   0
coupon_class                    0
commission_percent            368
current_amount_outstanding      0
cusip                         118
coupon_rate                     0
currency                        0
isin                            0
capital_tier                  368
seniority                       0
name                            0
sector                          0
trade_date                      0
bid_yield                       0
ask_yield                       0
high_yield                      1
low_yield                       1
dtype: int64

In [5]:
#commission_percent and capisnital_tier are null for all rows and can be droppe
df.drop(['commission_percent', 'capital_tier'], axis=1, inplace=True)


In [6]:
#Info on all the data types in our df
df.dtypes


Unnamed: 0                      int64
Unnamed: 0.1                    int64
security_id                     int64
amount                        float64
issue_date                     object
issue_yield                   float64
maturity_date                  object
coupon_class                   object
current_amount_outstanding    float64
cusip                          object
coupon_rate                   float64
currency                       object
isin                           object
seniority                      object
name                           object
sector                          int64
trade_date                     object
bid_yield                     float64
ask_yield                     float64
high_yield                    float64
low_yield                     float64
dtype: object

In [7]:
#Checking for duplicate values
df.duplicated().sum()


0

In [8]:
df.describe()


,Unnamed: 0,Unnamed: 0.1,security_id,amount,issue_yield,current_amount_outstanding,coupon_rate,sector,bid_yield,ask_yield,high_yield,low_yield
count,368.000000,368.000000,368.000000,3.680000e+02,246.000000,3.680000e+02,368.000000,3.680000e+02,368.000000,368.000000,367.000000,367.000000
mean,183.500000,183.500000,101673.891304,2.379517e+10,277.046401,2.794599e+10,3.273957,4.249232e+07,209.993804,206.406929,217.419619,206.837330
std,106.376689,106.376689,131788.137163,1.725329e+10,180.367984,2.055782e+10,1.859101,4.271337e+06,100.392680,99.215808,95.161330,99.209206
min,0.000000,0.000000,1140.000000,6.570000e+06,0.000000,0.000000e+00,0.875000,4.010101e+07,-408.700000,-433.200000,147.300000,-474.000000
25%,91.750000,91.750000,9692.250000,4.499999e+09,169.875000,3.414068e+09,1.875000,4.010101e+07,146.675000,145.997500,152.450000,145.600000
50%,183.500000,183.500000,26118.500000,2.893881e+10,230.050000,2.932534e+10,2.625000,4.010101e+07,171.800000,167.985000,176.900000,171.200000
75%,275.250000,275.250000,308972.250000,3.500003e+10,311.375000,3.911439e+10,4.450000,4.010101e+07,212.850000,203.412500,220.500000,208.900000
max,367.000000,367.000000,337554.000000,8.442707e+10,887.000000,7.511223e+10,8.750000,5.010102e+07,454.200000,449.200000,461.700000,449.200000


In [9]:
#Converting string date to datetime object
df['trade_date'] =  pd.to_datetime(df['trade_date'], format='%Y-%m-%d')
df['issue_date'] =  pd.to_datetime(df['issue_date'], format='%Y-%m-%d')


In [10]:
#Creating Tenor column using difference in trade_date and issue_date year
df['Tenor'] = pd.DatetimeIndex(df['trade_date']).year - pd.DatetimeIndex(df['issue_date']).year


In [11]:
#We can see that the max value of 29 is well above the mean and 75th percentile and it might be an outlier and will need to be checked 
df['Tenor'].describe()

count    368.000000
mean       5.214674
std        6.028552
min        0.000000
25%        2.000000
50%        3.000000
75%        6.000000
max       29.000000
Name: Tenor, dtype: float64

In [12]:
df['name'].unique()
df1 = df.loc[df['name'] == 'AT&T Inc']
df2 = df.loc[df['name'] == 'United States Treasury']

In [13]:
df2 = df2.groupby(['name','Tenor'])['bid_yield'].sum().reset_index()
df1 = df1.groupby(['name','Tenor'])['bid_yield'].sum().reset_index()


In [28]:
df1 = df1.loc[df1['Tenor'].isin([2,3,5,7,10,30])]
df2 = df2.loc[df2['Tenor'].isin([2,3,5,7,10,30])]


In [29]:
#Here i do a inner join to the the two df based on tenor number
#****I have assumed that if either the company or goverment bid_yield for a tenor is nan it will not be 0 but rather not counted
dff = pd.merge(df1, df2, left_index=True, right_index=True)


In [30]:
dff['Spread values'] = dff['bid_yield_x'] - dff['bid_yield_y']

In [31]:
dff.drop(['bid_yield_x', 'name_y', 'Tenor_y', 'bid_yield_y'], axis=1, inplace=True)


In [32]:
dff = dff.rename(columns={'name_x': 'company_name', 'Tenor_x': 'standard tenor'})


In [33]:
dff

,company_name,standard tenor,Spread values
3,AT&T Inc,3,750.70
5,AT&T Inc,5,-3180.00
7,AT&T Inc,7,-344.51
10,AT&T Inc,10,-505.20
